In [10]:
import cv2
import numpy as np
import os
import copy
import sys
import math
import torch
import matplotlib.pyplot as plt
import time
import yaml
from prodict import Prodict
import torch
import pdb

In [2]:
BASE_DIR = "/home/denso/carlos_vsr_workspace/mapfe4mp"
sys.path.append(BASE_DIR)

import model.datasets.argoverse.dataset_utils as dataset_utils
import model.datasets.argoverse.dataset as dataset
import model.datasets.argoverse.map_functions as map_functions 
import model.datasets.argoverse.goal_points_functions as goal_points_functions                                              

from argoverse.map_representation.map_api import ArgoverseMap

avm = ArgoverseMap()

INFO:argoverse.data_loading.vector_map_loader:Loaded root: ArgoverseVectorMap
INFO:argoverse.data_loading.vector_map_loader:Loaded root: ArgoverseVectorMap


# Load config

In [3]:
config_dir = BASE_DIR + "/config/config_social_lstm_mhsa.yml"
with open(config_dir) as config:
    config = yaml.safe_load(config)
    config = Prodict.from_dict(config)
    config.base_dir = BASE_DIR

# Load dataset

In [4]:
from model.datasets.argoverse.dataset import ArgoverseMotionForecastingDataset, seq_collate
from torch.utils.data import DataLoader

config.dataset.path = BASE_DIR + "/" + "data/datasets/argoverse/motion-forecasting/"

config.dataset.split = "val"
config.dataset.split_percentage = 0.01 # To generate the final results, must be 1 (whole split test)
config.dataset.batch_size = 1 # Better to build the h5 results file
config.dataset.num_workers = 0
config.dataset.class_balance = -1.0 # Do not consider class balance in the split test
config.dataset.shuffle = False

data_images_folder = config.dataset.path + config.dataset.split + "/data_images"

data = ArgoverseMotionForecastingDataset(dataset_name=config.dataset_name,
                                                 root_folder=config.dataset.path,
                                                 obs_len=config.hyperparameters.obs_len,
                                                 pred_len=config.hyperparameters.pred_len,
                                                 distance_threshold=config.hyperparameters.distance_threshold,
                                                 split=config.dataset.split,
                                                 split_percentage=config.dataset.split_percentage,
                                                 shuffle=config.dataset.shuffle,
                                                 class_balance=config.dataset.class_balance,
                                                 obs_origin=config.hyperparameters.obs_origin,
                                                 preprocess_data=config.dataset.preprocess_data,
                                                 save_data=config.dataset.save_data)

loader = DataLoader(data,
                            batch_size=config.dataset.batch_size,
                            shuffle=config.dataset.shuffle,
                            num_workers=config.dataset.num_workers,
                            collate_fn=seq_collate)

Loading .npy files as np data structures ...


# Load generator from checkpoint

In [5]:
from model.models.social_lstm_mhsa import TrajectoryGenerator

exp_name = "social_lstm_mhsa/exp12"
model_path = BASE_DIR + "/save/argoverse/" + exp_name + "/argoverse_motion_forecasting_dataset_0_with_model.pt"
checkpoint = torch.load(model_path)
generator = TrajectoryGenerator(h_dim=config.model.generator.encoder_lstm.hdim,
                                num_layers=config.model.generator.encoder_lstm.num_layers,
                                bidirectional=config.model.generator.encoder_lstm.bidirectional)
generator.load_state_dict(checkpoint.config_cp['g_best_state'])
generator.cuda() # Use GPU
generator.eval()

TrajectoryGenerator(
  (encoder): EncoderLSTM(
    (encoder): LSTM(2, 256, num_layers=2, bidirectional=True)
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
  )
  (lne): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (sattn): MultiHeadAttention(
    (attention): DotProductAttention(
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (W_q): Linear(in_features=256, out_features=256, bias=False)
    (W_k): Linear(in_features=256, out_features=256, bias=False)
    (W_v): Linear(in_features=256, out_features=256, bias=False)
    (W_o): Linear(in_features=256, out_features=256, bias=False)
  )
  (lnc): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (mlp_decoder_context): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (decoder): TemporalDecoderLSTM(
    (decoder): LSTM(16,

# Run test

In [11]:
from model.datasets.argoverse.dataset_utils import relative_to_abs_sgan

num_samples = 5
output_all = []

ade_list = []
fde_list = []

with torch.no_grad():
    for batch_index, batch in enumerate(loader):
        if batch_index > 99:
            break
        batch = [tensor.cuda() for tensor in batch]
        print("batch_index: ", batch_index)
        pdb.set_trace()
        (obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_obj,
            loss_mask, seq_start_end, frames, object_cls, obj_id, ego_origin, nsl,_) = batch

        predicted_traj = []
        agent_idx = torch.where(object_cls==1)[0].cpu().numpy()
        traj_real = torch.cat([obs_traj, pred_traj_gt], dim=0) + ego_origin.permute(1,0,2)
        predicted_traj.append(traj_real[:, agent_idx,:])
        #print(agent_idx.shape, agent_idx)
        for _ in range(num_samples):
            # Get predictions
            pred_traj_fake_rel = generator(obs_traj, obs_traj_rel, seq_start_end, agent_idx)

            agent_obj_id = obj_id[agent_idx]

            # Get predictions in absolute coordinates
            pred_traj_fake = relative_to_abs_sgan(pred_traj_fake_rel, obs_traj[-1,agent_idx, :]) # 30,1,2
            traj_fake = torch.cat([obs_traj[:,agent_idx, :], pred_traj_fake], dim=0) + ego_origin.permute(1,0,2) # 50,1,2
            predicted_traj.append(traj_fake)
        
        predicted_traj = torch.stack(predicted_traj, axis=0)
        predicted_traj = predicted_traj.cpu().numpy()
        #if batch_index == 61:
        #    print(f" {batch_index} - {nsl}")
        output_all.append(predicted_traj)

batch_index:  0
> /tmp/ipykernel_5993/3900009085.py(17)<cell line: 9>()
     15         pdb.set_trace()
     16         (obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_obj,
---> 17             loss_mask, seq_start_end, frames, object_cls, obj_id, ego_origin, nsl,_) = batch
     18 
     19         predicted_traj = []



# Show qualitative results

In [1]:
import matplotlib
ax = plt.gca()
# 12, 13, 14

file_id = 73 # From 1 to N files in the split

predicted_traj = output_all[file_id-1] 

# Validation

## Good example: 95
## Very challenging: 71
## Almost impossible:

# Train

## Good example:
## Very challenging:
## Almost impossible: 6

# filename = data_images_folder + "/" + str(file_id) + ".png"

# img = cv2.imread(filename)
# img = cv2.resize(img, dsize=(600,600))

print("predicted_traj ", predicted_traj.shape)
with plt.rc_context({'xtick.color': 'white', 'ytick.color': 'white'}):
    plt.grid(linestyle='-', linewidth=2)
    plt.plot(predicted_traj[0,0,0,0],predicted_traj[0,0,0,1],'-o',c='r') #starting point here
    plt.plot(predicted_traj[0,:21,:,0],predicted_traj[0,:21,:,1],'*',c='b')

    for i in range(predicted_traj.shape[0]):
        if i == 0:
            c = 'black'
        else:
            c= np.random.rand(3,)
        plt.plot(predicted_traj[i,20:,:,0],predicted_traj[i,20:,:,1],'*',c=c)#np.random.rand(3,)
    plt.xlabel('map_x_coord (m)')
    plt.ylabel('map_y_coord (m)')
    
    #plt.imshow(img)
    
    ax.set_aspect('equal')
    ax.figure.set_size_inches(10,10)
    plt.show()

NameError: name 'plt' is not defined